In [2]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import pickle

In [4]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nborders/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nborders/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
stop = set(stopwords.words('english'))
got_stop = ['[spoilers]', '[deleted]', 'nan', '[removed]', '[spoiler]', 'deleted', '[Spoilers', 'Main]', 'Spoilers', 'Main', 'removed', 'spoiler', '[All', 'Spoilers]', 'all', 'spoilers', '[NO SPOILER]', 'no spoiler', '[SPOLER]', '(Spoilers', 'Main)', 'spoler', '[Spoilers]', '[SPOILERS]', 'SPOILERS', 'MAIN]', 'MAIN', 'extended)', 'extended', 'Production)', 'Production', 'EXTENDED]', 'EXTENDED', '[NO', 'SPOILERS]', 'NO', '*Spoilers', 'Ahead*', 'Ahead', 'Extended]', 'Extended', '{SPOILER}', 'SPOILER', '[No', 'Spoiler]', 'No', 'Spoiler', '(NO', 'SPOILERS)', 'MAIN)', '(SPOILERS', '(no', 'spoilers)', '[SPOILERS', 'Extended)']
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [4]:
got_reddit = pickle.load( open( "./Data/got_reddit.pkl", "rb" ) )
asoiaf_reddit = pickle.load( open( "./Data/asoiaf_reddit.pkl", "rb" ) )
freefolk_reddit = pickle.load( open( "./Data/freefolk_reddit.pkl", "rb" ) )

In [9]:
def remove_all_punctuation(comment):
    punct_list = [i for i in string.punctuation]
    
    while re.findall(r'\\n', comment) != []:
        comment = comment.replace(re.search(r'\\n', comment)[0], '')
    while re.findall(r'\\t', comment) != []:
        comment = comment.replace(re.search(r'\\t', comment)[0], '')
    while re.findall(r'\\u.{4}', comment) != []:
        comment = comment.replace(re.search(r'\\u.{4}', comment)[0], '')
    
    for i in punct_list:
        comment = comment.replace(i, ' ')
    
    return comment

In [10]:
def remove_punctuation(comment):
    punct_list = [i for i in string.punctuation]
    for i in punct_list:
        comment = comment.replace(i, ' ')
    return comment

In [11]:
def trim_and_lower(comment):
    arr = comment.split()
    comment = " ".join(arr).lower()
    return comment

In [14]:
#def remove_nums(comment)

In [13]:
def remove_stops(comment):
    stop_free = [i for i in comment.split() if i not in stop]
    got_stop_free = " ".join([i for i in stop_free if i not in got_stop])
    return got_stop_free

In [15]:
def lemmatize(comment):
    return " ".join(lemma.lemmatize(word) for word in comment.split())

In [26]:
def remove_num(comment):
    return ''.join([char for char in comment if not char.isdigit()])

In [27]:
got_reddit['clean'] = got_reddit['body'].apply(remove_all_punctuation)
got_reddit['clean'] = got_reddit['clean'].apply(trim_and_lower)
got_reddit['clean'] = got_reddit['clean'].apply(remove_num)
got_reddit['clean'] = got_reddit['clean'].apply(remove_stops)
got_reddit['clean'] = got_reddit['clean'].apply(lemmatize)

asoiaf_reddit['clean'] = asoiaf_reddit['body'].apply(remove_all_punctuation)
asoiaf_reddit['clean'] = asoiaf_reddit['clean'].apply(trim_and_lower)
asoiaf_reddit['clean'] = asoiaf_reddit['clean'].apply(remove_num)
asoiaf_reddit['clean'] = asoiaf_reddit['clean'].apply(remove_stops)
asoiaf_reddit['clean'] = asoiaf_reddit['clean'].apply(lemmatize)

freefolk_reddit['clean'] = freefolk_reddit['body'].apply(remove_all_punctuation)
freefolk_reddit['clean'] = freefolk_reddit['clean'].apply(trim_and_lower)
freefolk_reddit['clean'] = freefolk_reddit['clean'].apply(remove_num)
freefolk_reddit['clean'] = freefolk_reddit['clean'].apply(remove_stops)
freefolk_reddit['clean'] = freefolk_reddit['clean'].apply(lemmatize)

In [34]:
#pickle.dump(freefolk_reddit, open("freefolk_reddit_ready_vector.pkl", "wb" ) )
#pickle.dump(asoiaf_reddit, open("soiaf_reddit_ready_vector.pkl", "wb" ) )
#pickle.dump(got_reddit, open("got_reddit_ready_vector.pkl", "wb" ) )

In [3]:
got = pickle.load( open( "./Data/got_reddit_ready_vector.pkl", "rb" ) )
#asoiaf = pickle.load( open( "./Data/soiaf_reddit_ready_vector.pkl", "rb" ) )
#freefolk = pickle.load( open( "./Data/freefolk_reddit_ready_vector.pkl", "rb" ) )

In [7]:
got['clean'][0]

'could rewatch episode one would watch can’t wait season i’m curious see episode would rewatch could watch pick chronological order winter coming baelor blackwater rain castamere mhysa lion rose mountain viper child hardhome mother’s mercy door battle bastard wind winter spoil war dragon wolf hard come list there’s many great episode people limited time budget rewatching hope helpful spark conversation'

In [1]:
# cleaning each of subreddits separately so we can remove words which appear too frequently and not often enough

In [4]:
#GOT

count_dict_GOT = {}

for doc in got['clean']:
    for word in doc.split():
        if word in count_dict_GOT.keys():
            count_dict_GOT[word] +=1
        else:
            count_dict_GOT[word] = 1

In [22]:
#count_dict_GOT

In [23]:
### These are helper functions not used in the current cleaning process, saved for reference to potentially be used in further cleaning efforts.

#low_value = 15
#for key in count_dict.keys():
    #if count_dict[key] < low_value: 
#corpus = [doc.split() for doc in got['clean']]
#for item in got['clean']:
    #got['clean_list'][item] = item.split()
    #corpus = [word for word in corpus if word not in bad_words]

In [5]:
low_value = 20
bad_words_GOT = [key for key in count_dict_GOT.keys() if count_dict_GOT[key] < low_value]

In [6]:
corpus = [doc.split() for doc in got['clean']]

In [7]:
#a list of lists where each document is a string broken into a list of words
cleaned_corpus_GOT = []
for document in corpus:
    cleaned_corpus_GOT.append([word for word in document if word not in bad_words_GOT])

In [8]:
pickle.dump(cleaned_corpus_GOT, open("cleaned_corpus_GOT.pkl", "wb" ) )

In [9]:
extra_stop_words = ['ign', 'diggory', 'cedric', 'timon', 'pumba', 'arnold', 'gotzmilk', 'episode', 'season', 'character', 'show', 
                    'amp', 'xb', 'de', 'et', 'le', 'http', 'la', 'com', 'il', 'de', 'gt', 'se', 'gt', 'qui', 'lt', 'xb', 'amp',
                   'one', 'would', 'could', 'gif', "youl'", 'may', 'get', 'instagram', 'aa',
 'aaa',
 'aaaaaaand',
 'aaaaand',
 'aaaand',
 'aaand',
 'aabns',
 'aabxbmaea',
 'aafdadceaf',
 'aarambha',
 'aaron',
 'aarya',
 'ab', 'abc', 'er', 'nk', 'ww' ]



In [10]:
further_cleaned_corpus_GOT = []
for document in cleaned_corpus_GOT:
    further_cleaned_corpus_GOT.append([word for word in document if word not in extra_stop_words])

In [12]:
pickle.dump(further_cleaned_corpus_GOT, open("further_cleaned_corpus_GOT.pkl", "wb" ) )

In [13]:
final_GOT = []

for item in further_cleaned_corpus_GOT:
    item = ' '.join(item)
    final_GOT.append(item)

In [15]:
pickle.dump(final_GOT, open("final_got.pkl", "wb" ))